In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows',   5000)

In [ ]:
for i in range(1,51):
    s1= "site%da = pd.read_csv('./TrainingData/Fault1_Train%d', header=None)"%(i,i)
    s2= "site%dc = pd.read_csv('./TrainingData/Fault2_Train%d', header=None)"%(i,i)
    exec(s1)
    exec(s2)
    
    if i%10 == 0:
        print(i)

In [ ]:
site1a[0]

In [ ]:
site1c

In [ ]:
def rms(x):
    return np.sqrt(np.mean(x**2))

In [ ]:
NoOfSensor  = 4
NoOfFeature = 5
NoOfData    = 50

FeatureData_Fault1 = np.zeros((NoOfSensor*NoOfFeature , NoOfData))
FeatureData_Fault2 = np.zeros((NoOfSensor*NoOfFeature , NoOfData))
# 특징추출 코드 작성

for i in range(NoOfData):
    
    s1 = 'temp_data1=site%da'%(i+1) 
    s2 = 'temp_data2=site%dc'%(i+1)
    exec(s1)
    exec(s2)
    for j in range(NoOfSensor):
        
        FeatureData_Fault1[5*j+0, i] = np.max(temp_data1.iloc[:,j])
        FeatureData_Fault1[5*j+1, i] = np.min(temp_data1.iloc[:,j])
        FeatureData_Fault1[5*j+2, i] = np.mean(temp_data1.iloc[:,j])
        FeatureData_Fault1[5*j+3, i] = rms(temp_data1.iloc[:,j])
        FeatureData_Fault1[5*j+4, i] = np.var(temp_data1.iloc[:,j])
        
        FeatureData_Fault2[5*j+0, i] = np.max(temp_data2.iloc[:,j])
        FeatureData_Fault2[5*j+1, i] = np.min(temp_data2.iloc[:,j])
        FeatureData_Fault2[5*j+2, i] = np.mean(temp_data2.iloc[:,j])
        FeatureData_Fault2[5*j+3, i] = rms(temp_data2.iloc[:,j])
        FeatureData_Fault2[5*j+4, i] = np.var(temp_data2.iloc[:,j])

        
print(FeatureData_Fault1.shape)
print(FeatureData_Fault2.shape) 

In [ ]:
FeatureData_Fault1_df = pd.DataFrame(FeatureData_Fault1)
FeatureData_Fault1_df

In [ ]:
FeatureData_Fault2_df = pd.DataFrame(FeatureData_Fault2)
FeatureData_Fault2_df

In [ ]:
import seaborn           as sb
import scipy.stats       as sp

In [ ]:
NoOfData    = int(FeatureData_Fault1_df.shape[1])  
NoOfFeature = int(FeatureData_Fault1_df.shape[0])

P_value = np.zeros((NoOfFeature , 2))

for i in np.arange(NoOfFeature):
    
    T_test       = np.array(sp.ttest_ind(FeatureData_Fault1_df.iloc[i,:] , FeatureData_Fault2_df.iloc[i,:]))
    P_value[i,0] = i          
    P_value[i,1] = T_test[1]  
    
P_value      = pd.DataFrame(P_value)
P_value

In [ ]:
P_value_Rank = P_value.sort_values([1],ascending=True)  
P_value_Rank

In [ ]:
path = './MLmodel/P_value_Rank'  
P_value_Rank.to_csv(path, sep=',', header=None, index=None)

In [ ]:
Rank =   14     

FeatureSelected_Fault1 = np.zeros((Rank,NoOfData))
FeatureSelected_Fault2 = np.zeros((Rank,NoOfData))


for i in range(Rank):
    
    index         = int(P_value_Rank.iloc[i,0])
    FeatureSelected_Fault1[i,:]   = FeatureData_Fault1_df.iloc[index,:].values
    FeatureSelected_Fault2[i,:] = FeatureData_Fault2_df.iloc[index,:].values

FeatureSelected = pd.DataFrame(np.concatenate([FeatureSelected_Fault1, FeatureSelected_Fault2] , axis=1))

print(FeatureSelected.shape)  

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
NoOfData   = int(FeatureSelected.shape[1]/2)   # 데이터 개수 (정상/고장 각각) 각각 180개씩 
Fold       = 7                   # Fold 개수 선정 : 데이터 개수의 약수여야 함.

FeatNo     = int(FeatureSelected.shape[0])  # 데이터 특징 수 (=데이터 차원)
FoldDataNo = int(NoOfData/Fold)            # 1개 Fold 당 (검증)데이터 개수

print(FeatNo)
print(FoldDataNo)

In [ ]:
NormalSet   = np.array(FeatureSelected.iloc[: , :NoOfData])
AbnormalSet = np.array(FeatureSelected.iloc[: , NoOfData:])
FeatureSelected_Reshaped = pd.DataFrame(np.concatenate([NormalSet , AbnormalSet] , axis=0))
FeatureSelected_Reshaped.shape

In [ ]:
# Validation Data set
for i in range(Fold):
    
    temp_Valid_Normal   = FeatureSelected_Reshaped.iloc[:FeatNo , FoldDataNo*i : FoldDataNo*(i+1)]
    temp_Valid_Abnormal = FeatureSelected_Reshaped.iloc[FeatNo: , FoldDataNo*i : FoldDataNo*(i+1)]
    temp_Valid = pd.DataFrame(np.transpose(np.concatenate([temp_Valid_Normal, temp_Valid_Abnormal] , axis=1)))
    
    s = 'Validation_Fold%d = temp_Valid'%(i+1)
    exec(s)

    
# Training Data set
for i in range(Fold):
    
    temp_Train_Front = FeatureSelected_Reshaped.iloc[:,:FoldDataNo*i]
    temp_Train_Back  = FeatureSelected_Reshaped.iloc[:,FoldDataNo*(i+1):]
    temp_Train_Total = np.concatenate([temp_Train_Front , temp_Train_Back] , axis=1)
    temp_Train_Final = pd.DataFrame(np.transpose(np.concatenate([temp_Train_Total[:FeatNo,:],temp_Train_Total[FeatNo:,:]] , axis=1)))
    
    s ='Training_Fold%d  = temp_Train_Final'%(i+1)
    exec(s)


In [ ]:
Validation_Fold1.shape

In [ ]:
Training_Fold1.shape

In [ ]:
NoOfLabel_Train = int(Training_Fold1.shape[0]/2)
NoOfLabel_Valid = int(Validation_Fold1.shape[0]/2)

print(NoOfLabel_Train)
print(NoOfLabel_Valid)

In [ ]:
TrainingFold_Label   = np.zeros(2*NoOfLabel_Train , dtype=int)
ValidationFold_Label = np.zeros(2*NoOfLabel_Valid , dtype=int)

# 고장데이터(학습용) Label 값 = 1
TrainingFold_Label[NoOfLabel_Train:] = 1

# 고장데이터(검증용) Label 값 = 1
ValidationFold_Label[NoOfLabel_Valid:] = 1

TrainingFold_Label   = pd.Series(TrainingFold_Label)
ValidationFold_Label = pd.Series(ValidationFold_Label)

#TrainingFold_Label
ValidationFold_Label

In [ ]:
Training_All = np.transpose(FeatureSelected)

Training_All.shape

In [ ]:
Training_All_Label = np.zeros(NoOfData*2)

Training_All_Label[NoOfData:] = 1    # 고장데이터(학습용) Label 값 = 1
Training_All_Label = pd.Series(Training_All_Label)

Training_All_Label.shape

In [ ]:
Add    = 0
Divide = 0

for i in range(Fold):
    c1 = 'Training_CurrentFold = Training_Fold%d'%(i+1)
    exec(c1)
    c2 = 'Validation_CurrentFold = Validation_Fold%d'%(i+1)
    exec(c2)    

    knnModel_CurrentFold = KNeighborsClassifier(n_neighbors = 1).fit(Training_CurrentFold , TrainingFold_Label)
    
    c3 = 'knnscore_Fold%d = knnModel_CurrentFold.score(Validation_CurrentFold , ValidationFold_Label)'%(i+1)
    exec(c3)
        
    Add += knnModel_CurrentFold.score(Validation_CurrentFold, ValidationFold_Label)
    Divide += 1
    
Avg_accuracy = Add/Divide

print('[Result of K-fold Cross Validation] \n')
print(' Fold 1: {:.2f}% \n Fold 2: {:.2f}% \n Fold 3: {:.2f}% \n Fold 4: {:.2f}% \n Fold 5: {:.2f}% \n Fold 6: {:.2f}% \n Fold 7: {:.2f}%'.
        format(knnscore_Fold1*100, knnscore_Fold2*100, knnscore_Fold3*100, knnscore_Fold4*100, knnscore_Fold5*100,knnscore_Fold6*100,knnscore_Fold7*100))
print('\n Average accuracy: {:.2f}%'.format(Avg_accuracy*100))

In [ ]:
KNN_model = KNeighborsClassifier(n_neighbors = 1).fit(Training_All , Training_All_Label)

joblib.dump(KNN_model, './MLmodel/KNN_model.plk')

In [ ]:
KNN_model_Final = joblib.load('./MLmodel/KNN_model.plk')

In [ ]:
Add    = 0
Divide = 0

for i in range(Fold):
    c1 = 'Validation_CurrentFold = Validation_Fold%d'%(i+1)
    exec(c1)    
    
    c2 = 'knnscore_Fold%d = KNN_model_Final.score(Validation_CurrentFold , ValidationFold_Label)'%(i+1)
    exec(c2)
        
    Add += KNN_model_Final.score(Validation_CurrentFold, ValidationFold_Label)
    Divide += 1
    
Avg_accuracy = Add/Divide

print('[Result of K-fold Cross Validation] \n')
print(' Fold 1: {:.2f}% \n Fold 2: {:.2f}% \n Fold 3: {:.2f}% \n Fold 4: {:.2f}% \n Fold 5: {:.2f}% \n Fold 6: {:.2f}% \n Fold 7: {:.2f}%'.
        format(knnscore_Fold1*100, knnscore_Fold2*100, knnscore_Fold3*100, knnscore_Fold4*100, knnscore_Fold5*100,knnscore_Fold6*100,knnscore_Fold7*100))
print('\n Average accuracy: {:.2f}%'.format(Avg_accuracy*100))